# Обучение с учителем
# Прогнозирование ухода клиента из банка в ближайшее время
## Описание проекта
Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.  

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.  

Постройте модель с предельно большим значением F1-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте F1-меру на тестовой выборке самостоятельно.  

Дополнительно измеряйте AUC-ROC, сравнивайте её значение с F1-мерой.  

Источник данных: https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling

## Описание данных
Признаки
- **RowNumber** — индекс строки в данных
- **CustomerId** — уникальный идентификатор клиента
- **Surname** — фамилия
- **CreditScore** — кредитный рейтинг
- **Geography** — страна проживания
- **Gender** — пол
- **Age** — возраст
- **Tenure** — количество недвижимости у клиента
- **Balance** — баланс на счёте
- **NumOfProducts** — количество продуктов банка, используемых клиентом
- **HasCrCard** — наличие кредитной карты
- **IsActiveMember** — активность клиента
- **EstimatedSalary** — предполагаемая зарплата

Целевой признак
- **Exited** — факт ухода клиента

## Шаг 1. Загрузите и подготовьте данные. Поясните порядок действий.

In [1]:
# Импорт нужных библиотек и классов
import pandas as pd
import numpy as np
# Разделение на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split
import joblib
# Алгоритм классификации - решающее дерево
from sklearn.tree import DecisionTreeClassifier
# Алгоритм классификации - случайный лес
from sklearn.ensemble import RandomForestClassifier
# Алгоритм классификации - логистическая регрессия
from sklearn.linear_model import LogisticRegression
# Механизм GridSearchCV
from sklearn.model_selection import GridSearchCV
# Механизм StratifiedKFold
from sklearn.model_selection import StratifiedKFold
# Метрика F1
from sklearn.metrics import f1_score
# Метрика AUC-ROC
from sklearn.metrics import roc_auc_score
# Метрика "Точность"
from sklearn.metrics import precision_score
# Метрика "Полнота"
from sklearn.metrics import recall_score
# Инструмент перемешивания объектов
from sklearn.utils import shuffle

In [2]:
churn_df = pd.read_csv('churn.csv')

In [3]:
churn_df.info()
churn_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Из описания видно, что:
 1. Можно избавиться от столбца **RowNumber**, т.к. его значения содержатся в индексе.
 2. Можно избавиться от столбца **Surname**, т.к. он в нашей задаче не нужен.
 3. Переименовать столбцы в привычную нотацию
 4. Переименовать **Geography** => **country**, **HasCrCard** => **has_credit_card**, **IsActiveMember** => **is_active**, **NumOfProducts** => **product_count**, **Exited** => **is_exited**
 4. С помощью OHE-кодирования избавиться от столбцов **country** и **female**
 5. Убрать из рассмотрения строки, где tenure равен null, т.к. не совсем понятно как заполнить данный пропуск.

In [4]:
churn_df = churn_df.drop('RowNumber', axis=1)
churn_df = churn_df.drop('Surname', axis=1)
churn_df.columns = ['customer_id', 'credit_score', 'country', 'gender', 'age', 'tenure', 'balance', 'product_count', 'has_credit_card', 'is_active', 'estimated_salary', 'is_exited']
churn_df = pd.get_dummies(churn_df, drop_first=True)
churn_df = churn_df[~churn_df['tenure'].isnull()]

In [5]:
churn_df.head()

,customer_id,credit_score,age,tenure,balance,product_count,has_credit_card,is_active,estimated_salary,is_exited,country_Germany,country_Spain,gender_Male
0,15634602,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,15647311,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,15619304,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,15701354,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,15737888,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


## Шаг 2. Исследуйте баланс классов, обучите модель без учёта дисбаланса. Кратко опишите выводы.

In [6]:
churn_df_zeros = churn_df[churn_df['is_exited'] == 0]
churn_df_ones = churn_df[churn_df['is_exited'] == 1]
print('churn_df.shape = ', churn_df.shape)
print('churn_df_zeros.shape = ', churn_df_zeros.shape)
print('churn_df_ones.shape = ', churn_df_ones.shape)

churn_df.shape =  (9091, 13)
churn_df_zeros.shape =  (7237, 13)
churn_df_ones.shape =  (1854, 13)


Видим, что объектов положительного класса примерно в 4 раза меньше, чем объектов отрицательного класса. Попробуем обучить модель без учёта дисбаланса классов.  

### Подготовка данных к обучению
Сначала выделим features и target признаки, а затем разделим исходные данные на обучающую, валидационную и тестовую выборки.

In [7]:
# Сначала выделим features и target признаки
features = churn_df.drop(['is_exited'], axis=1)
target = churn_df['is_exited']

Делить исходные данные будем в пропорциях: Обучающая выборка 60%, Валидационная 20% и Тестовая 20%
Для этого воспользуемся методом train_test_split из библиотеки sklearn. Она делит исходные данные на 2 части, следовательно применять её надо 2 раза.
Сначала поделим выделим тестовую выборку (20% от исх.), а потом из большего остатка (80% от исх.) выделим обучающую и валидационную выборки (по 75% и 25% от остатка соответственно).

In [8]:
features_comb, features_test, target_comb, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)
features_train, features_valid, target_train, target_valid = train_test_split(features_comb, target_comb, test_size=0.25, random_state=12345)

### Подбор гиперпараметров моделей
Попробуем подобрать оптимальные настройки моделей с помощью механизма GridSearchCV.

In [9]:
# Подбор параметра max_depth для модели решающего дерева
dtc = DecisionTreeClassifier(random_state=12345)
param_grid = { 
    'max_depth' : list(range(1, 10)),
}
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
CV_dtc = GridSearchCV(estimator=dtc, param_grid=param_grid, cv= k_fold, scoring = 'f1')
CV_dtc.fit(features_train, target_train)
print(CV_dtc.best_params_)
print(CV_dtc.best_score_)
print(CV_dtc.best_estimator_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: Undefined

{'max_depth': 7}
0.5616553162962769
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')


In [10]:
# Подбор параметров n_estimators и max_depth для модели случайного леса
rfc = RandomForestClassifier(random_state=12345)
param_grid = { 
    'n_estimators': list(range(2, 21, 2)),
    'max_depth' : list(range(2, 17, 2)),
}
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= k_fold, scoring = 'f1')
# Операция занимает долгое время, поэтому следующие команды закоментированы по умолчанию
# (результат подбора: {'max_depth': 10, 'n_estimators': 18})
#CV_rfc.fit(features_train, target_train)
#print(CV_rfc.best_params_)
#print(CV_rfc.best_score_)
#print(CV_rfc.best_estimator_)

Таким образом, для модели решающего дерева самая лучшая максимальная глубина равна 7, а для модели случайного леса макс. глубина равна 10, кол-во деревьев - 18.

### Расчёт метрик F1 и AUC-ROC для найденных моделей решающего дерева и случайного леса, а также для логистической регрессии

In [11]:
# Модель решающего дерева
model = DecisionTreeClassifier(max_depth=7, random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('DecisionTreeClassifier model has f1 =', '%.3f'%f1, 'and auc_roc =', '%.3f'%auc_roc)

DecisionTreeClassifier model has f1 = 0.568 and auc_roc = 0.819


In [12]:
# Модель случайного леса
model = RandomForestClassifier(max_depth=10, n_estimators=18, random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('RandomForestClassifier model has f1 =', '%.3f'%f1, 'and auc_roc =', '%.3f'%auc_roc)

RandomForestClassifier model has f1 = 0.513 and auc_roc = 0.831


In [13]:
# Модель логистической регрессии
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
precision = precision_score(target_valid, predicted_valid)
recall = recall_score(target_valid, predicted_valid)
f1 = f1_score(target_valid, predicted_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('LogisticRegression model has f1 =', f1, 'precision =', precision, 'recall =', recall, 'and auc_roc =', '%.3f'%auc_roc)

LogisticRegression model has f1 = 0.0 precision = 0.0 recall = 0.0 and auc_roc = 0.564


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


## Вывод
По метрике F1 лучше всех справилась модель решающего дерева. По метрике AUC-ROC лучше всех оказалась модель случайного леса. Но видим, что до 1 обеим метрикам ещё далеко.  
Логистическая регрессия вообще не смогла предсказать ни одного объекта с положительным классом и метрика F1 у нее равна 0.  

Попробуем улучшить модели за счёт балансировки классов.

## Шаг 3. Улучшите качество модели, учитывая дисбаланс классов. Обучите разные модели и найдите лучшую. Кратко опишите выводы.

### Взвешивание классов
Начнём с технологии взвешивания классов, для этого в моделях надо указать параметр **class_weight='balanced'**.  
Сначала подберём гиперпараметры для моделей решающего дерева и случайного леса, а затем проверим обученные модели на валидационной выборке.

In [14]:
# Подбор параметра max_depth для модели решающего дерева
dtc = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
param_grid = { 
    'max_depth' : list(range(1, 10)),
}
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
CV_dtc = GridSearchCV(estimator=dtc, param_grid=param_grid, cv= k_fold, scoring = 'f1')
CV_dtc.fit(features_train, target_train)
print(CV_dtc.best_params_)
print(CV_dtc.best_score_)
print(CV_dtc.best_estimator_)

# Подбор параметров n_estimators и max_depth для модели случайного леса
rfc = RandomForestClassifier(random_state=12345, class_weight='balanced')
param_grid = { 
    'n_estimators': list(range(2, 31, 2)),
    'max_depth' : list(range(2, 17, 2)),
}
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= k_fold, scoring = 'f1')
# Операция занимает долгое время, поэтому следующие команды закоментированы по умолчанию 
# (результат подбора: {'max_depth': 10, 'n_estimators': 20})
#CV_rfc.fit(features_train, target_train)
#print(CV_rfc.best_params_)
#print(CV_rfc.best_score_)
#print(CV_rfc.best_estimator_)

{'max_depth': 5}
0.5756024524503323
DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')


In [15]:
# Модель решающего дерева
model = DecisionTreeClassifier(max_depth=5, random_state=12345, class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
joblib.dump(model, 'dtr_model_bal.joblib')
print('DecisionTreeClassifier model has f1 =', '%.2f'%f1, 'and auc_roc =', '%.2f'%auc_roc)

# Модель случайного леса
model = RandomForestClassifier(max_depth=10, n_estimators=20, random_state=12345, class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
joblib.dump(model, 'rfc_model_bal.joblib')
print('RandomForestClassifier model has f1 =', '%.2f'%f1, 'and auc_roc =', '%.2f'%auc_roc)

# Модель логистической регрессии
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
precision = precision_score(target_valid, predicted_valid)
recall = recall_score(target_valid, predicted_valid)
f1 = f1_score(target_valid, predicted_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
joblib.dump(model, 'lr_model_bal.joblib')
print('LogisticRegression model has f1 =', '%.2f'%f1, 'and auc_roc =', '%.2f'%auc_roc)

DecisionTreeClassifier model has f1 = 0.54 and auc_roc = 0.81
RandomForestClassifier model has f1 = 0.59 and auc_roc = 0.84
LogisticRegression model has f1 = 0.48 and auc_roc = 0.76


### Вывод
По метрикам F1 и AUC-ROC лучше всех справилась модель случайного леса, с ней мы достигли значения F1=0.59, которое требовалось в задании проекта.  
Модель логистической регрессии улучшила свои показатели и теперь метрика F1 у неё равна 0.48.  
Попытаемся сделать наши модели ещё лучше за счёт увеличения выборки.

### Увеличение выборки
Т.к. объктов положительного класса примерно в 4 раза меньше, чем объектов отрицательного класса, то попробуем увеличить кол-во объектов положительного класса в 4 раза, и тогда должен наступить баланс классов.

In [16]:
# Функция увеличения выборки
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [17]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [18]:
# Подбор параметра max_depth для модели решающего дерева
dtc = DecisionTreeClassifier(random_state=12345)
param_grid = { 
    'max_depth' : list(range(1, 10)),
}
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
CV_dtc = GridSearchCV(estimator=dtc, param_grid=param_grid, cv= k_fold, scoring = 'f1')
CV_dtc.fit(features_upsampled, target_upsampled)
print(CV_dtc.best_params_)
print(CV_dtc.best_score_)
print(CV_dtc.best_estimator_)

# Подбор параметров n_estimators и max_depth для модели случайного леса
rfc = RandomForestClassifier(random_state=12345)
param_grid = { 
    'n_estimators': list(range(2, 31, 2)),
    'max_depth' : list(range(2, 17, 2)),
}
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= k_fold, scoring = 'f1')
# Операция занимает долгое время, поэтому следующие команды закоментированы по умолчанию 
# (результат подбора: {'max_depth': 16, 'n_estimators': 28})
#CV_rfc.fit(features_upsampled, target_upsampled)
#print(CV_rfc.best_params_)
#print(CV_rfc.best_score_)
#print(CV_rfc.best_estimator_)

{'max_depth': 9}
0.8313530324288924
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=9,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')


In [19]:
# Модель решающего дерева
model = DecisionTreeClassifier(max_depth=9, random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
joblib.dump(model, 'dtr_model_up.joblib')
print('DecisionTreeClassifier model has f1 =', '%.2f'%f1, 'and auc_roc =', '%.2f'%auc_roc)

# Модель случайного леса
model = RandomForestClassifier(max_depth=16, n_estimators=28, random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
joblib.dump(model, 'rfc_model_up.joblib')
print('RandomForestClassifier model has f1 =', '%.2f'%f1, 'and auc_roc =', '%.2f'%auc_roc)

# Модель логистической регрессии
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
precision = precision_score(target_valid, predicted_valid)
recall = recall_score(target_valid, predicted_valid)
f1 = f1_score(target_valid, predicted_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
joblib.dump(model, 'lr_model_up.joblib')
print('LogisticRegression model has f1 =', '%.2f'%f1, 'and auc_roc =', '%.2f'%auc_roc)

DecisionTreeClassifier model has f1 = 0.54 and auc_roc = 0.78
RandomForestClassifier model has f1 = 0.57 and auc_roc = 0.84
LogisticRegression model has f1 = 0.35 and auc_roc = 0.56


### Вывод
Как видим увеличение выборки не помогло улучшить значение метрики F1. Попробуем воспользоваться уменьшением выборки.
### Уменьшение выборки
Т.к. объктов положительного класса примерно в 4 раза меньше, чем объектов отрицательного класса, то попробуем оставить одну четверть от объектов отрицательного класса, и тогда должен наступить баланс классов.

In [20]:
# Функция уменьшения выборки
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [21]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

In [22]:
# Подбор параметра max_depth для модели решающего дерева
dtc = DecisionTreeClassifier(random_state=12345)
param_grid = { 
    'max_depth' : list(range(1, 10)),
}
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
CV_dtc = GridSearchCV(estimator=dtc, param_grid=param_grid, cv= k_fold, scoring = 'f1')
CV_dtc.fit(features_downsampled, target_downsampled)
print(CV_dtc.best_params_)
print(CV_dtc.best_score_)
print(CV_dtc.best_estimator_)

# Подбор параметров n_estimators и max_depth для модели случайного леса
rfc = RandomForestClassifier(random_state=12345)
param_grid = { 
    'n_estimators': list(range(2, 31, 2)),
    'max_depth' : list(range(2, 17, 2)),
}
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= k_fold, scoring = 'f1')
# Операция занимает долгое время, поэтому следующие команды закоментированы по умолчанию 
# (результат подбора: {'max_depth': 6, 'n_estimators': 24})
#CV_rfc.fit(features_downsampled, target_downsampled)
#print(CV_rfc.best_params_)
#print(CV_rfc.best_score_)
#print(CV_rfc.best_estimator_)

{'max_depth': 4}
0.7460979428403552
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')


In [23]:
# Модель решающего дерева
model = DecisionTreeClassifier(max_depth=4, random_state=12345)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
joblib.dump(model, 'dtr_model_down.joblib')
print('DecisionTreeClassifier model has f1 =', '%.2f'%f1, 'and auc_roc =', '%.2f'%auc_roc)

# Модель случайного леса
model = RandomForestClassifier(max_depth=6, n_estimators=24, random_state=12345)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
joblib.dump(model, 'rfc_model_down.joblib')
print('RandomForestClassifier model has f1 =', '%.2f'%f1, 'and auc_roc =', '%.2f'%auc_roc)

# Модель логистической регрессии
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
precision = precision_score(target_valid, predicted_valid)
recall = recall_score(target_valid, predicted_valid)
f1 = f1_score(target_valid, predicted_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
joblib.dump(model, 'lr_model_down.joblib')
print('LogisticRegression model has f1 =', '%.2f'%f1, 'and auc_roc =', '%.2f'%auc_roc)

DecisionTreeClassifier model has f1 = 0.49 and auc_roc = 0.79
RandomForestClassifier model has f1 = 0.57 and auc_roc = 0.85
LogisticRegression model has f1 = 0.35 and auc_roc = 0.57


### Вывод
Уменьшение выборки не позволило добиться увеличения значения метрики F1.  
Таким образом, получаем, что самые лучшие модели - это модели случайного леса. Все три варианта этой модели имеют примерно одинакоовое значение метрики F1 - поэтому проведём финальное тестирование на тестовой выборке для всех 3 вариантов.

## Шаг 4. Проведите финальное тестирование.

In [24]:
def rfc_test(model_name):
    model = joblib.load(model_name)
    predicted_test = model.predict(features_test)
    f1 = f1_score(target_test, predicted_test)
    probabilities_test = model.predict_proba(features_test)
    probabilities_one_test = probabilities_test[:, 1]
    auc_roc = roc_auc_score(target_test, probabilities_one_test)
    print('RandomForestClassifier', model_name, 'has f1 =', '%.2f'%f1, 'and auc_roc =', '%.2f'%auc_roc)

rfc_test('rfc_model_bal.joblib')
rfc_test('rfc_model_up.joblib')
rfc_test('rfc_model_down.joblib')

RandomForestClassifier rfc_model_bal.joblib has f1 = 0.57 and auc_roc = 0.85
RandomForestClassifier rfc_model_up.joblib has f1 = 0.59 and auc_roc = 0.85
RandomForestClassifier rfc_model_down.joblib has f1 = 0.58 and auc_roc = 0.84


### Вывод
На тестовой выборке мы также получили примерно одинаковые значения по метрикам F1 и AUC-ROC. Также в ходе выполнения проекта было замечено, что метрика AUC-ROC всегда превосходит метрику F1.